In [11]:
import hopsworks

project = hopsworks.login()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14502


In [ ]:
!python3 -m streamlit run streamlit_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.20.30:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14502
Connected. Call `.close()` to terminate connection gracefully.
2023-04-21 23:14:28.262 USE `test5_featurestore`
2023-04-21 23:14:28.787 WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`city_name` `city_name`, `fg1`.`date` `date`, `fg1`.`pm2_5` `pm2_5`, `fg1`.`pm_2_5_previous_1_day` `pm_2_5_previous_1_day`, `fg1`.`pm_2_5_previous_2_day` `pm_2_5_previous_2_day`, `fg1`.`pm_2_5_previous_3_day` `pm_2_5_previous_3_day`, `fg1`.`pm_2_5_previous_4_day` `pm_2_5_previous_4_day`, `fg1`.`pm_2_5_previous_5_day` `pm_2_5_previous_5_day`, `fg1`.`pm_2_5_previous_6_day` `pm_2_5_previous_6_day`, `fg1`.`pm_2_5_previous

In [1]:
continent = "EU"